Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Setup Dataset

In [ ]:
!pip install kagglehub
import kagglehub

path = kagglehub.dataset_download("cookiefinder/tomato-disease-multiple-sources")

!mv /root/.cache/kagglehub/datasets/cookiefinder/tomato-disease-multiple-sources/versions/ /content/drive/MyDrive/tomato


100%|██████████| 1.37G/1.37G [00:17<00:00, 82.3MB/s]

Extracting files...


Set the dataset directories

In [5]:
import os
from PIL import Image

def check_and_remove_corrupted_images(directory):
    for subdir, dirs, files in os.walk(directory):
        for file in files:
            try:
                img_path = os.path.join(subdir, file)
                with Image.open(img_path) as img:
                    img.verify()
            except (IOError, SyntaxError) as e:
                print(f"Corrupted image: {img_path}")
                os.remove(img_path)

train_dir = '/content/drive/MyDrive/tomato/1/train'
valid_dir = '/content/drive/MyDrive/tomato/1/valid'


In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from PIL import ImageFile

ImageFile.LOAD_TRUNCATED_IMAGES = True

Data Generators

In [7]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=8,
    class_mode='categorical'
)

valid_data = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(150, 150),
    batch_size=8,
    class_mode='categorical'
)


Found 25851 images belonging to 11 classes.
Found 6683 images belonging to 11 classes.


Build the Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.layers import GlobalAveragePooling2D

model = Sequential([
    Input(shape=(150, 150, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    GlobalAveragePooling2D(),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(train_data.class_indices), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


Add Checkpoints

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint_path = '/content/drive/MyDrive/checkpoints/best_model.keras'
checkpoint = ModelCheckpoint(checkpoint_path, save_best_only=True, monitor='val_loss', mode='min')

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


Train the Model

In [9]:
from tensorflow.keras.models import load_model

model = load_model(checkpoint_path)


last_completed_epoch = 4
total_epochs = 5

history = model.fit(
    train_data,
    validation_data=valid_data,
    epochs=total_epochs,
    initial_epoch=last_completed_epoch,
    callbacks=[early_stopping, checkpoint]
)


Epoch 5/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3232/3232 ━━━━━━━━━━━━━━━━━━━━ 9216s 3s/step - accuracy: 0.6390 - loss: 1.0290 - val_accuracy: 0.7432 - val_loss: 0.7229


Resume Training

Save Model and Logs

In [10]:
model.save('/content/drive/MyDrive/final_model.keras')